In [525]:
# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
from numpy import loadtxt
import pandas as pd
from tqdm import tqdm
from pprint import pprint
import requests
import json
import csv
import math
from datetime import datetime
from dateutil.relativedelta import relativedelta
import statistics

# Import dependencies
import numpy as np
import matplotlib.pyplot as plt
from numpy import loadtxt
import pandas as pd
from tqdm import tqdm
from pprint import pprint
import requests
import json
import csv
import math
from datetime import datetime
from dateutil.relativedelta import relativedelta
import statistics

# Importing sector, price, financials, & market data

In [526]:
# Get the organCode from list of all tickers
def process(filename:str="/Users/nambuismbp/projects/investment_research/data/codedict.txt") -> dict:
    ticker_to_organ_code = {}
    with open(filename, "r") as json_file:
        data = json.load(json_file)
        ticker_to_organ_code = {
            item["ticker"]: item["organCode"] for item in data["items"]}
        return ticker_to_organ_code
    return None

ticker_dict = process()

# Get ticker from the organCode
def get_ticker(val):
    for key, value in ticker_dict.items():
        if val == value:
            return key

In [527]:
# Load in sector data
dfSectorInfo = pd.read_csv(r'/Users/nambuismbp/projects/investment_research/data/descriptive_info.csv')

In [528]:
# Load in pricing data
dfPricingInfo = pd.read_csv(r'/Users/nambuismbp/projects/investment_research/data/pricing_data.csv')

# Uniform column name to be ticker
for col in dfPricingInfo:
    dfPricingInfo.rename(columns = {col: get_ticker(col)}, inplace = True)

# Create new monthly date series and insert into dataframe
monthSeries = pd.date_range('2008-01-31', periods=len(dfPricingInfo), freq='M')
count = 0
for date in reversed(range(len(monthSeries))):
    dfPricingInfo.at[count, 'date'] = monthSeries[date]
    count = count + 1

# Rearrange columns so that date appears first
del dfPricingInfo[dfPricingInfo.columns[0]]
cols = dfPricingInfo.columns.tolist()
cols = cols[-1:] + cols[:-1]
dfPricingInfo = dfPricingInfo[cols]

In [529]:
# Load in trading value data
dfTradingVal = pd.read_csv(r'/Users/nambuismbp/projects/investment_research/data/trading_value_data.csv')

# Put trading value data having the same dates as pricing data
del dfTradingVal[dfTradingVal.columns[0]]
extracted = dfPricingInfo['date'].to_list()
dfTradingVal.insert(0, "date", extracted)

# Uniform column name to be ticker
for col in dfTradingVal:
    if col != 'date': dfTradingVal.rename(columns = {col: get_ticker(col)}, inplace = True)

In [530]:
# Load in financial data
with open("/Users/nambuismbp/projects/investment_research/data/financial_data_dec29.txt", 'r') as file:
    financials_raw = json.load(file)

financials={}
for (key, value) in financials_raw.items():
    ticker = get_ticker(key)
    financials[ticker] = value

In [531]:
# Load in market return
dfMarketReturn = pd.read_csv(r'/Users/nambuismbp/projects/investment_research/data/market_return.csv')
monthSeries = pd.date_range('2000-07-31', periods=len(dfMarketReturn), freq='M')
count = 0
for date in reversed(range(len(monthSeries))):
    dfMarketReturn.at[count, 'date'] = monthSeries[date]
    count = count + 1

# Scale up returns to match display format of stock & portfolios
dfMarketReturn['market_return'] = dfMarketReturn['market_return'].multiply(100)

# Functions to select the right universe based on data availability & liquidity


In [532]:
# Select only companies that has all 4 quarters of financial data in the previous year
def check_financial_data(date, data) -> list:
    financialScreeningList = []
    for ticker in data:
        count = data[ticker]['years'].count(date.year - 1)
        if count == 4: financialScreeningList.append(ticker)
    return financialScreeningList

In [533]:
# Select only companies that have pricing data in the month
def check_pricing_data(date, data) -> list:
    priceScreeningList = []
    index = data[data['date'] == datetime(date.year, date.month, date.day)].index[0]
    for ticker in dfPricingInfo:
        if ticker != 'date' and not math.isnan(dfPricingInfo.loc[index, ticker]):
            priceScreeningList.append(ticker)
    return priceScreeningList

In [534]:
# Select only companies that are not banks, insurance or financial services
def check_sector_data(dataframe) -> list:
    dataRemove = dataframe[dataframe['ICB_Sector_L2'] != 'Dịch vụ tài chính L2']
    dataRemove = dataRemove[dataRemove['ICB_Sector_L2'] != 'Ngân hàng L2']
    dataRemove = dataRemove[dataRemove['ICB_Sector_L2'] != 'Bảo hiểm L2']
    
    sectorScreeningList = dataRemove['Ticker'].to_list()
    return sectorScreeningList

# Functions to rank and sort stocks by factors


In [535]:
def get_marketcap(date, tickers, financials) -> dict:
    quarter = (date.month-1)//3 + 1
    sizeFactorDict = {}
    for ticker in tickers:
        # Getting current Market Cap
        currDateIndex = 0
        currNumShare = 0
        numShare = 0
        marketCap = 0
        for i in range(len(financials[ticker]['years'])):
            if financials[ticker]['years'][i] == date.year and financials[ticker]['quarters'][i] == quarter:
                currDateIndex = i
        if currDateIndex != 0:
            numShare = financials[ticker]['num_share'][currDateIndex]
        
        currPriceIndex = dfPricingInfo[dfPricingInfo['date'] == date].index[0]
        currPrice = dfPricingInfo.loc[currPriceIndex, ticker]
        
        if numShare != 0 and currPrice != 0 and not np.isnan(currPrice) and not np.isnan(numShare):
            marketCap = numShare * currPrice
            sizeFactorDict[ticker] = marketCap
    return sizeFactorDict


In [536]:
def get_size(date, tickers, financials) -> dict:
    quarter = (date.month-1)//3 + 1
    sizeFactorDict = {}
    for ticker in tickers:
        # Getting current Market Cap
        currDateIndex = 0
        currNumShare = 0
        numShare = 0
        marketCap = 0
        for i in range(len(financials[ticker]['years'])):
            if financials[ticker]['years'][i] == date.year and financials[ticker]['quarters'][i] == quarter:
                currDateIndex = i
        if currDateIndex != 0:
            numShare = financials[ticker]['num_share'][currDateIndex]
        
        currPriceIndex = dfPricingInfo[dfPricingInfo['date'] == date].index[0]
        currPrice = dfPricingInfo.loc[currPriceIndex, ticker]
        
        if numShare != 0 and currPrice != 0 and not np.isnan(currPrice) and not np.isnan(numShare):
            marketCap = numShare * currPrice
            sizeFactorDict[ticker] = 1 / math.log(marketCap)
    return sizeFactorDict


# Functions to create decile portfolios and calculate returns

In [537]:
def create_decile_portfolios(sortedDict, decile) -> list:
    keys = pd.Series(list(sortedDict.keys()))
    values = pd.Series(list(sortedDict.values()))
    df = pd.DataFrame({'ticker': keys, 'values': values})
    df['decile_rank'] = pd.qcut(df['values'], numPortfolio, labels = False)
    
    idx = df.index[df['decile_rank'] == decile].tolist()
    decileList = df['ticker'][idx].tolist()
    return decileList

In [538]:
def diff_month(d1, d2) -> int:
    return (d2.year - d1.year) * 12 + d2.month - d1.month

#Function to calculate average monthly return of a stock in month beginmonth + 1
def get_stock_monthly_return(stock, beginmonth) -> float:
    index = dfPricingInfo.index[dfPricingInfo['date'] == beginmonth]
    
    beginPrice = dfPricingInfo.iloc[index][stock].iat[0]
    endPrice = dfPricingInfo.iloc[index-1][stock].iat[0]
    
    monthlyReturn = (endPrice / beginPrice - 1) * 100
    return monthlyReturn

In [539]:
# Equal-weighted average returns of stocks in portfolio
def get_portfolio_monthly_return_EW(portfolio, beginPeriod, endPeriod) -> float:
    monthSeries = pd.date_range(start=beginPeriod, end=endPeriod, freq = 'M')
    portfolioMonthlyReturn = []
    
    for month in monthSeries:
        if month == endPeriod:
            break
        temp = []
        for stock in portfolio:
            temp.append(get_stock_monthly_return(stock, month))
        average = sum(temp) / len(temp)
        portfolioMonthlyReturn.append(average)
                    
    return portfolioMonthlyReturn

In [540]:
# Value-weighted (by market cap) average returns of stocks in portfolio
def get_portfolio_monthly_return_VW(portfolio, beginPeriod, endPeriod) -> float:
    monthSeries = pd.date_range(start=beginPeriod, end=endPeriod, freq = 'M')
    portfolioMonthlyReturn = []
    
    for month in monthSeries:
        if month == endPeriod:
            break
        stockretList = []
        marketcapList = []
        
        size = get_marketcap(month, portfolio, financials)
        
        for stock in portfolio:
            if stock in size:
                stockretList.append(get_stock_monthly_return(stock, month))
                marketcapList.append(size[stock])
        
        weights = []
        for stock in marketcapList:
            weights.append(stock / sum(marketcapList))
        
        weightedavg = sum(ret * weights for ret, weights in zip(stockretList, weights))
        portfolioMonthlyReturn.append(weightedavg)
        
    return portfolioMonthlyReturn

In [541]:
def get_stock_monthly_trading_val(stock, beginmonth) -> float:
    index = dfTradingVal.index[dfTradingVal['date'] == beginmonth]
    tradingVal = dfTradingVal.iloc[index-1][stock].iat[0]
    return tradingVal

In [542]:
def get_portfolio_yearly_avg_trading_val(portfolio, beginPeriod, endPeriod) -> float:
    monthSeries = pd.date_range(start=beginPeriod, end=endPeriod, freq = 'M')
    portfolioTradingVal = []
    
    for month in monthSeries:
        if month == endPeriod:
            break
        temp = []
        for stock in portfolio:
            temp.append(get_stock_monthly_trading_val(stock, month))
        average = sum(temp) / len(temp)
        portfolioTradingVal.append(average)
    return sum(portfolioTradingVal) / len(portfolioTradingVal)

# Functions to get beta & standard deviation

In [543]:
def get_portfolio_beta(portfolioReturns, market_data) -> list:
    betaList = []
    monthlyRet = []
    marketRet = []
    
    #Calculate beta
    for portfolio in range(numPortfolio):
        tempList = []
        for year in portfolioReturns:
            tempList.extend(portfolioReturns[year][portfolio])
        monthlyRet.append(tempList)

    t = beginDate
    for period in range(len(yearSeries)):
        index = dfMarketReturn[dfMarketReturn['date'] == t].index[0]
        index = index - 1
        for i in range(12):
            marketRet.append(dfMarketReturn.loc[index, 'market_return'])
            index = index - 1 
        t = t.replace(year = t.year + 1)
    
    for i in range(len(monthlyRet)):
        dfMonthlyRet = pd.Series(monthlyRet[i])
        dfMarketRet = pd.Series(marketRet)
        covar = dfMonthlyRet.cov(dfMarketRet)
        var = dfMarketRet.var()
        beta = round(covar / var, 3)
        betaList.append(beta)
    return betaList

In [544]:
def get_portfolio_std(portfolioReturns) -> list:
    monthlyRet = []
    stdList = []
    for portfolio in range(numPortfolio):
        tempList = []
        for year in portfolioReturns:
            tempList.extend(portfolioReturns[year][portfolio])
        monthlyRet.append(tempList)
    for i in range(len(monthlyRet)):
        dfMonthlyRet = pd.Series(monthlyRet[i])
        stdList.append(round(dfMonthlyRet.std(), 3))
    return stdList

# Set parameters for the model

In [545]:
# Set begin and end dates for backtesting
beginDate = datetime(2012, 6, 30)
endDate = datetime(2022, 6, 30)

# Set number of portfolios
numPortfolio = 10

# Set liquidity restriction by value
minLiquidity = 50000000

# Running model

In [546]:
# Generate date series for the backtesting
yearSeries = pd.date_range(start=beginDate, end=endDate, freq = 'Y')

In [547]:
# Get the universe of acceptable stocks for each year
date = beginDate
universe = {}
for year in yearSeries:
    accountingYear = date - relativedelta(years=1)
    financialScreeningList = check_financial_data(accountingYear, financials)
    priceScreeningList = check_pricing_data(date, dfPricingInfo)
    sectorScreeningList = check_sector_data(dfSectorInfo)
    universe[year] = list(set(sectorScreeningList) & set(priceScreeningList) & set(financialScreeningList))    
    
    print(f'In yearly period from {date.year} to {date.year + 1} the investable unvierse has {len(universe[year])} stocks')
    date = date.replace(year = date.year + 1)

In yearly period from 2012 to 2013 the investable unvierse has 190 stocks
In yearly period from 2013 to 2014 the investable unvierse has 219 stocks
In yearly period from 2014 to 2015 the investable unvierse has 232 stocks
In yearly period from 2015 to 2016 the investable unvierse has 245 stocks
In yearly period from 2016 to 2017 the investable unvierse has 251 stocks
In yearly period from 2017 to 2018 the investable unvierse has 272 stocks
In yearly period from 2018 to 2019 the investable unvierse has 289 stocks
In yearly period from 2019 to 2020 the investable unvierse has 307 stocks
In yearly period from 2020 to 2021 the investable unvierse has 329 stocks
In yearly period from 2021 to 2022 the investable unvierse has 344 stocks


In [548]:
# Get the decile portfolios based on ranking factor for each year
portfolios = {}

for year in yearSeries:
    keys = universe[year]
    # Get a dictionary of the value of factor for each stock
    factor = get_size(datetime(year.year, beginDate.month, beginDate.day), keys, financials)
    
    # Remove stocks from the ranking list if they are not in the universe
    factor = {key:val for key, val in factor.items() if key in universe[year]}
    
    # Remove stocks that do not meet liquidity restriction of more than 10 million VND per day
    liquidityScreening = []
    for stock in keys:
        yearlyLiquidity = []
        for month in range(12):
            yearlyLiquidity.append(get_stock_monthly_trading_val(stock, datetime(year.year, beginDate.month, beginDate.day)))
        avgMonthlyLiquidity = (sum(yearlyLiquidity) / len(yearlyLiquidity)) / 12
        avgDailyLiquidity = avgMonthlyLiquidity / 21
        if avgDailyLiquidity > minLiquidity:
            liquidityScreening.append(stock)
            
    # Remove stocks from the ranking list if they do not meet liquidity restrictions
    factor = {key:val for key, val in factor.items() if key in liquidityScreening}
    
    # Sort based on factor
    factorSorted = dict(sorted(factor.items(), key=lambda item: item[1], reverse = True))
    
    # List of decile portfolios in a given year
    portfolioList = []
    countStock = []
    for decile in range(numPortfolio):
        getPortfolio = create_decile_portfolios(factorSorted, decile)
        portfolioList.append(getPortfolio)
        countStock.extend(getPortfolio)
        
        avgScore = []
        
        # Print out average BE/ME for each decile in the year
        for item in getPortfolio:
            avgScore.append(factor[item])
        #print(f"The average BE/ME in year {year.year} for portfolio decile {decile + 1} is {round(sum(avgScore)/len(avgScore), 3)}")
        #print(f"Number of companies in year {year.year} for portfolio decile {decile + 1} is {len(getPortfolio)}")        

    print(f"In the period between {year.year} and {year.year + 1}, there are {len(countStock)} companies in total with an average of {len(countStock)/numPortfolio} in each portfolio")
    portfolios[year] = portfolioList

In the period between 2012 and 2013, there are 79 companies in total with an average of 7.9 in each portfolio
In the period between 2013 and 2014, there are 85 companies in total with an average of 8.5 in each portfolio
In the period between 2014 and 2015, there are 114 companies in total with an average of 11.4 in each portfolio
In the period between 2015 and 2016, there are 127 companies in total with an average of 12.7 in each portfolio
In the period between 2016 and 2017, there are 154 companies in total with an average of 15.4 in each portfolio
In the period between 2017 and 2018, there are 169 companies in total with an average of 16.9 in each portfolio
In the period between 2018 and 2019, there are 137 companies in total with an average of 13.7 in each portfolio
In the period between 2019 and 2020, there are 160 companies in total with an average of 16.0 in each portfolio
In the period between 2020 and 2021, there are 183 companies in total with an average of 18.3 in each portfo

In [549]:
# Examine the average trading value for each decile portfolio in a given year
for year in yearSeries:
    beginPeriod = datetime(year.year, beginDate.month, beginDate.day)
    endPeriod = datetime(year.year+1, beginDate.month-1, beginDate.day)
    for decile in range(numPortfolio):
        myport = portfolios[year][decile]
        portfolioTradingVal = get_portfolio_yearly_avg_trading_val(myport, beginPeriod, endPeriod)
        stockDailyTradingVal = (portfolioTradingVal / 21) / len(myport)
        print(f"From year {year.year} to {year.year+1}, the average daily trading value for a stock in portfolio decile {decile+1} was {round(stockDailyTradingVal, 0):,}")

From year 2012 to 2013, the average daily trading value for a stock in portfolio decile 1 was 2,374,068,497.0
From year 2012 to 2013, the average daily trading value for a stock in portfolio decile 2 was 1,374,972,649.0
From year 2012 to 2013, the average daily trading value for a stock in portfolio decile 3 was 759,237,618.0
From year 2012 to 2013, the average daily trading value for a stock in portfolio decile 4 was 732,107,513.0
From year 2012 to 2013, the average daily trading value for a stock in portfolio decile 5 was 737,486,538.0
From year 2012 to 2013, the average daily trading value for a stock in portfolio decile 6 was 766,680,531.0
From year 2012 to 2013, the average daily trading value for a stock in portfolio decile 7 was 249,868,236.0
From year 2012 to 2013, the average daily trading value for a stock in portfolio decile 8 was 201,553,145.0
From year 2012 to 2013, the average daily trading value for a stock in portfolio decile 9 was 166,234,152.0
From year 2012 to 2013, 

From year 2019 to 2020, the average daily trading value for a stock in portfolio decile 8 was 208,583,906.0
From year 2019 to 2020, the average daily trading value for a stock in portfolio decile 9 was 176,702,693.0
From year 2019 to 2020, the average daily trading value for a stock in portfolio decile 10 was 207,102,725.0
From year 2020 to 2021, the average daily trading value for a stock in portfolio decile 1 was 8,730,476,507.0
From year 2020 to 2021, the average daily trading value for a stock in portfolio decile 2 was 4,064,806,268.0
From year 2020 to 2021, the average daily trading value for a stock in portfolio decile 3 was 2,845,580,133.0
From year 2020 to 2021, the average daily trading value for a stock in portfolio decile 4 was 1,995,408,009.0
From year 2020 to 2021, the average daily trading value for a stock in portfolio decile 5 was 979,023,814.0
From year 2020 to 2021, the average daily trading value for a stock in portfolio decile 6 was 1,147,798,788.0
From year 2020 to

# Equal weighted portfolios

In [550]:
# Get equal weighted monthly returns for each portfolio for each year
date = beginDate

portfolioReturnsEW = {}
for year in yearSeries: # go inside year, having access to all portfolios
    nextYear = date.replace(year = date.year + 1)
    returnsInYear = []

    for i in range(len(portfolios[year])): # go inside to get monthly returns of individual portfolio
        temp = get_portfolio_monthly_return_EW(portfolios[year][i], date, nextYear)
        returnsInYear.append(temp)
    portfolioReturnsEW[year] = returnsInYear
    
    # Move to next year
    date = nextYear

In [551]:
# Calculate average returns of all months during testing period for decile portfolios
avgMonthlyPortfolioReturnsEW = []

for i in range(numPortfolio):
    allYearsAvg = []
    for year in portfolioReturnsEW:
        oneYearAvg = sum(portfolioReturnsEW[year][i]) / len(portfolioReturnsEW[year][i])
        allYearsAvg.append(oneYearAvg)
        print(f"In year {year.year}, portfolio decile {i+1} has an average monthly return of {round(oneYearAvg, 3)} percent")
    
    avgMonthlyPortfolioReturnsEW.append(sum(allYearsAvg) / len(allYearsAvg))

In year 2012, portfolio decile 1 has an average monthly return of 0.915 percent
In year 2013, portfolio decile 1 has an average monthly return of 1.554 percent
In year 2014, portfolio decile 1 has an average monthly return of -0.448 percent
In year 2015, portfolio decile 1 has an average monthly return of -0.179 percent
In year 2016, portfolio decile 1 has an average monthly return of 0.735 percent
In year 2017, portfolio decile 1 has an average monthly return of 1.171 percent
In year 2018, portfolio decile 1 has an average monthly return of 0.902 percent
In year 2019, portfolio decile 1 has an average monthly return of -1.054 percent
In year 2020, portfolio decile 1 has an average monthly return of 3.766 percent
In year 2021, portfolio decile 1 has an average monthly return of -0.485 percent
In year 2012, portfolio decile 2 has an average monthly return of 1.885 percent
In year 2013, portfolio decile 2 has an average monthly return of 2.147 percent
In year 2014, portfolio decile 2 has

In [552]:
avgMonthlyPortfolioBetaEW = get_portfolio_beta(portfolioReturnsEW, dfMarketReturn)
avgMonthlyPortfolioStdEW = get_portfolio_std(portfolioReturnsEW)

# Value weighted portfolios

In [553]:
# Get value weighted monthly returns for each portfolio for each year
date = beginDate

portfolioReturnsVW = {}
for year in yearSeries: # go inside year, having access to all portfolios
    nextYear = date.replace(year = date.year + 1)
    returnsInYear = []

    for i in range(len(portfolios[year])): # go inside to get monthly returns of individual portfolio
        temp = get_portfolio_monthly_return_VW(portfolios[year][i], date, nextYear)
        returnsInYear.append(temp)
    portfolioReturnsVW[year] = returnsInYear
    
    # Move to next year
    date = nextYear

In [554]:
# Calculate average returns of all months during testing period for decile portfolios
avgMonthlyPortfolioReturnsVW = []

for i in range(numPortfolio):
    allYearsAvg = []
    for year in portfolioReturnsVW:
        oneYearAvg = sum(portfolioReturnsVW[year][i]) / len(portfolioReturnsVW[year][i])
        allYearsAvg.append(oneYearAvg)
        print(f"In year {year.year}, portfolio decile {i+1} has an average monthly return of {round(oneYearAvg, 3)} percent")
    
    avgMonthlyPortfolioReturnsVW.append(sum(allYearsAvg) / len(allYearsAvg))

In year 2012, portfolio decile 1 has an average monthly return of 1.115 percent
In year 2013, portfolio decile 1 has an average monthly return of 0.74 percent
In year 2014, portfolio decile 1 has an average monthly return of -1.614 percent
In year 2015, portfolio decile 1 has an average monthly return of 0.929 percent
In year 2016, portfolio decile 1 has an average monthly return of 1.072 percent
In year 2017, portfolio decile 1 has an average monthly return of 3.248 percent
In year 2018, portfolio decile 1 has an average monthly return of 0.464 percent
In year 2019, portfolio decile 1 has an average monthly return of -1.736 percent
In year 2020, portfolio decile 1 has an average monthly return of 3.256 percent
In year 2021, portfolio decile 1 has an average monthly return of -1.214 percent
In year 2012, portfolio decile 2 has an average monthly return of 1.424 percent
In year 2013, portfolio decile 2 has an average monthly return of 1.776 percent
In year 2014, portfolio decile 2 has a

In [555]:
avgMonthlyPortfolioBetaVW = get_portfolio_beta(portfolioReturnsVW, dfMarketReturn)
avgMonthlyPortfolioStdVW = get_portfolio_std(portfolioReturnsVW)

# Printing out the results

In [558]:
names = []
for portfolio in range(numPortfolio):
    names.append(f"Decile {str(portfolio+1)}")

dfReturnsEW = pd.DataFrame({'Value Factor': names, 'Avg Monthly Return': avgMonthlyPortfolioReturnsEW, 'Stdev': avgMonthlyPortfolioStdEW})
dfReturnsVW = pd.DataFrame({'Value Factor': names, 'Avg Monthly Return': avgMonthlyPortfolioReturnsVW, 'Stdev': avgMonthlyPortfolioStdVW})
print("Average monthly returns of equal weighted portfolios for SIZE factor")
print(dfReturnsEW)
print()
print("Average monthly returns of value weighted portfolios SIZE factor")
print(dfReturnsVW)

Average monthly returns of equal weighted portfolios for SIZE factor
  Value Factor  Avg Monthly Return  Stdev
0     Decile 1            0.687685  6.242
1     Decile 2            1.418470  7.206
2     Decile 3            1.455404  6.693
3     Decile 4            1.093278  7.283
4     Decile 5            0.981757  6.096
5     Decile 6            1.813661  7.073
6     Decile 7            1.981247  7.455
7     Decile 8            1.917698  7.531
8     Decile 9            2.038140  7.875
9    Decile 10            2.984154  8.218

Average monthly returns of value weighted portfolios SIZE factor
  Value Factor  Avg Monthly Return  Stdev
0     Decile 1            0.626088  6.036
1     Decile 2            1.328504  7.249
2     Decile 3            1.401060  6.759
3     Decile 4            1.080148  7.305
4     Decile 5            1.086807  6.097
5     Decile 6            1.934133  7.120
6     Decile 7            2.109907  7.591
7     Decile 8            1.791547  7.965
8     Decile 9           

In [557]:
# Cumulative returns of value weighted portfolio
dfCumReturns = pd.DataFrame()
for portfolio in range(numPortfolio):
    data = []
    for year in portfolioReturnsVW: 
        for month in portfolioReturnsVW[year][portfolio]:
            data.append(month)
    dfMonthlyReturns["Portfolio " + str(portfolio + 1)] = pd.Series(data)
        
    marketMonthlyReturn = []
    t = beginDate
    for period in range(len(yearSeries)):
        index = dfMarketReturn[dfMarketReturn['date'] == t].index[0]
        index = index - 1
        for i in range(12):
            marketMonthlyReturn.append(dfMarketReturn.loc[index, 'market_return'])
            index = index - 1 
        t = t.replace(year = t.year + 1)
        
    cumReturn = []
    cumReturnMarket = []
        
    totalReturn = 100
    totalMarketReturn = 100
    for i in range(len(data)):
        totalReturn = math.ceil(totalReturn * (1 + data[i]/100))
        totalMarketReturn = math.ceil(totalMarketReturn * (1 + marketMonthlyReturn[i]/100))
        cumReturn.append(totalReturn)
        cumReturnMarket.append(totalMarketReturn)
        
    dfCumReturns["Decile " + str(portfolio + 1)] = pd.Series(cumReturn)
dfCumReturns["VNIndex"] = pd.Series(cumReturnMarket)

dfCumReturns['months'] = pd.date_range(start=beginDate.replace(month = beginDate.month + 1), end=endDate, freq = 'M')
dfCumReturns = dfCumReturns.set_index('months')
print(dfCumReturns)

            Decile 1  Decile 2  Decile 3  Decile 4  Decile 5  Decile 6  \
months                                                                   
2012-07-31        98        99        97        94       103       104   
2012-08-31        98        92        94        86        98        96   
2012-09-30        98        82        90        80        95        92   
2012-10-31        98        82        93        85        90        93   
2012-11-30        94        81        91        84        90        98   
...              ...       ...       ...       ...       ...       ...   
2022-02-28       301       632       616       493       504      1257   
2022-03-31       298       672       652       533       531      1407   
2022-04-30       282       569       573       463       476      1236   
2022-05-31       276       543       541       422       442      1106   
2022-06-30       261       482       539       376       399       967   

            Decile 7  Decile 8  Decil